In [ ]:

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score,
    f1_score, roc_auc_score, confusion_matrix,
    precision_recall_curve, roc_curve
)
from imblearn.over_sampling import SMOTE
import matplotlib.pyplot as plt
import warnings, joblib, os
warnings.filterwarnings("ignore")


df = pd.read_csv("creditcard.csv")
print("Dataset shape:", df.shape)
print("Class distribution:\n", df['Class'].value_counts())

# Drop 'Time', scale 'Amount'
if 'Time' in df.columns:
    df.drop('Time', axis=1, inplace=True)
scaler = StandardScaler()
df['Amount'] = scaler.fit_transform(df[['Amount']])

X, y = df.drop('Class', axis=1), df['Class']

X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)


sm = SMOTE(random_state=42)
X_res, y_res = sm.fit_resample(X_train, y_train)
print("\nAfter SMOTE balancing:\n", pd.Series(y_res).value_counts())

rf = RandomForestClassifier(
    n_estimators=150,
    max_depth=10,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_res, y_res)

# Select top features based on importance
importances = pd.DataFrame({
    'Feature': X.columns,
    'Importance': rf.feature_importances_
}).sort_values(by='Importance', ascending=False)

top_features = importances['Feature'].head(12).tolist()
print("\nTop Selected Features:\n", top_features)

X_train_sel = X_res[top_features]
X_test_sel = X_test[top_features]

log_reg = LogisticRegression(max_iter=2000, solver='liblinear', random_state=42)

param_grid = {
    'C': [0.1, 1, 5],
    'penalty': ['l1', 'l2']
}
grid = GridSearchCV(log_reg, param_grid, scoring='f1', cv=3, n_jobs=-1)
grid.fit(X_train_sel, y_res)
best_model = grid.best_estimator_
print("\nBest Parameters Found:", grid.best_params_)
y_proba = best_model.predict_proba(X_test_sel)[:, 1]
precisions, recalls, thresholds = precision_recall_curve(y_test, y_proba)
f1_scores = 2 * (precisions[:-1] * recalls[:-1]) / (precisions[:-1] + recalls[:-1])
best_thr = thresholds[np.argmax(f1_scores)]
print(f"\nOptimal Threshold for Best F1: {round(best_thr, 3)}")
y_pred = (y_proba >= best_thr).astype(int)
acc = accuracy_score(y_test, y_pred)
prec = precision_score(y_test, y_pred)
rec = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_proba)
cm = confusion_matrix(y_test, y_pred)
print("\n Final Model Performance ")
print(f"Accuracy : {acc*100:.3f}%")
print(f"Precision: {prec*100:.3f}%")
print(f"Recall   : {rec*100:.3f}%")
print(f"F1-score : {f1*100:.3f}%")
print(f"AUC-ROC  : {auc*100:.3f}%")
print("\nConfusion Matrix:\n", cm)
flagged_frauds = X_test.copy()
flagged_frauds['Actual'] = y_test
flagged_frauds['Predicted'] = y_pred
suspicious = flagged_frauds[flagged_frauds['Predicted'] == 1]
print("\n🔍 Suspicious Transactions Detected by the Model:")
print(suspicious.head(10))  # show first 10 frauds
fpr, tpr, _ = roc_curve(y_test, y_proba)
plt.figure(figsize=(6,5))
plt.plot(fpr, tpr, color='blue', label=f"AUC = {auc:.4f}")
plt.plot([0, 1], [0, 1], 'r--')
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve - Logistic Regression (Top Features)")
plt.legend()
plt.grid(True)
plt.tight_layout()
os.makedirs("fraud_detection_model", exist_ok=True)
plt.savefig("fraud_detection_model/roc_curve.png", dpi=200)
plt.close()
os.makedirs("fraud_detection_model", exist_ok=True)
joblib.dump(best_model, "fraud_detection_model/hybrid_model.pkl")
joblib.dump(scaler, "fraud_detection_model/scaler.pkl")
print("\n Model and Scaler saved in 'fraud_detection_model' folder.")
df_features = df.drop('Class', axis=1).copy()
df_features['Amount'] = scaler.transform(df_features[['Amount']])
df_selected = df_features[top_features]
y_proba_all = best_model.predict_proba(df_selected)[:, 1]
y_pred_all = (y_proba_all >= best_thr).astype(int)
all_results = df.copy()
all_results['Predicted'] = y_pred_all
detected_frauds = all_results[all_results['Predicted'] == 1]
print(f"\nTotal Fraud Transactions in full dataset: {df['Class'].sum()}")
print(f"Total Fraud Transactions Detected (Full Data): {len(detected_frauds)}")

Dataset shape: (284807, 31)
Class distribution:
 Class
0    284315
1       492
Name: count, dtype: int64

After SMOTE balancing:
 Class
0    227451
1    227451
Name: count, dtype: int64
